# Analisis de posibilidad de join entre datos sernapesca

### Import libreries

In [21]:
import logging
import subprocess
import os   
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta

In [18]:
%%time
reporte_plantas_raw = pd.read_excel("../data/datos_sernapesca/Reporte de Plantas.xlsx")
ingresos_raw = pd.read_excel("../data/datos_sernapesca/Ingresos 2021-2022.xlsx")
RNA_raw = pd.read_excel("../data/datos_sernapesca/RNA.XLSX")

CPU times: user 30.3 s, sys: 68 ms, total: 30.4 s
Wall time: 30.4 s


### Filtrando por región y tipo de producto

In [19]:
# Se filtra por region, no es necesario hacer filtro por producto ya que solo se encuentra 
# info disponible para moluscos
reporte_plantas_filtered = reporte_plantas_raw[(reporte_plantas_raw['Región Bodega'] == 'X REGION') |
                                        (reporte_plantas_raw['Región Agente'] == 'X REGION') |
                                        (reporte_plantas_raw['Tipo Ult. Proveedor'] == 'CC')]
reporte_plantas_filtered = reporte_plantas_filtered.reset_index()          
# columnmas a eliminar.
to_drop = ['Región Agente',
           'Fecha Declaración',
           'Nº Declaración',
           'idStock',
           'Tipo Item',
           'Estado Recurso',
           'Folio Origen',
           'Tipo Ult. Proveedor',
           'Folio Ult. Proveedor',
           'utilizado',
           'Unidades Rest',
           'Unidades Abastecidas',
           'Tipo Declaración',
           'idBodega',
           'Nombre Bodega']
# eliminando columnas que no sirven.
reporte_plantas_filtered = reporte_plantas_filtered.drop(to_drop, axis=1)


# Se filtra sólo por tipo de especie = molusco, ya que este dataset ya era sólo de la décima región
ingresos_filtered = ingresos_raw[ingresos_raw['Tipo Especie'] == 'MOLUSCOS']
# columnmas a eliminar.
to_drop = ['ACS',
            'Región',
            'N° Declaración',
            'Tipo Especie',
            'Sistema Ingreso']
# eliminando columnas que no sirven.
ingresos_filtered = ingresos_filtered.drop(to_drop, axis=1)


# El registro nacional de acuicultura de sernapesca (RNA) se encuentra para todo chile, por lo que se filtra
# para la décima región. También se filtra por aquellos centros que se encuentran inscritos.
RNA_filtered = RNA_raw[(RNA_raw['Región'] == 10) & 
                       (RNA_raw['Categoria'] == 'INSCRITO')]

In [ ]:
shape_reportes =  reporte_plantas_filtered.shape
shape_ingresos =  ingresos_filtered.shape
shape_RNA =  RNA_filtered.shape
print(f'''El dataset de reportes filtrado tiene una forma {shape_reportes}, 
        mientras que el de ingresos es de la forma {shape_ingresos} y 
        el de RNA es de {shape_RNA}''')

El dataset de reportes filtrado tiene una forma (53974, 20), 
        mientras que el de ingresos es de la forma (7998, 17) y 
        el de RNA es de (2338, 18)


Se tienen 3 datasets y queremos saber si es que es posible hacer join entre los 3. Para esto y en primera instancia es necesario definir los campos que podrían ser utilizados para el join. 

* Por parte de la tabla de ingresos, código de agente parece ser una buena opción y también código del último proovedor.

* Para el caso de la tabla de reportes, existe el codigo de centro y el codigo de origen, los cuales podrían servir.

* Finalmente la tabbla de RNA contiene el código del centro de cultivo acuícola.

### Cúal es la producción actual de las plantas?

In [ ]:
reporte_plantas_filtered
ingresos_filtered
RNA_filtered

In [30]:
date_to_consult = '2022-06-30'
days_to_track = 30
date_to_consult = datetime.strptime(date_to_consult, '%Y-%m-%d')
start_date =  date_to_consult - timedelta(days=days_to_track)
reporte_plantas_last_month = reporte_plantas_filtered[(reporte_plantas_filtered['Fecha Ult. Proveedor'] > start_date) & 
                                                      (reporte_plantas_filtered['Fecha Ult. Proveedor'] < '2022-06-30')]
reporte_plantas_last_month_sum = reporte_plantas_last_month[['Código Agente','Toneladas Abastecidas']].groupby('Código Agente').sum()
reporte_plantas_last_month_sum

,Toneladas Abastecidas
Código Agente,
3061,14.372
3150,9.343
4153,18.836
5251,7.441
8320,2.670
8476,12.000
10034,2039.300
10213,47.762
10217,3302.865


In [31]:
ingresos_filtered


,Codigo Centro,Propietario,Nombre Sector,Tipo Cultivo,Tipo Agua,Año,Mes,Provincia,Comuna,Cod. Especie,Especie,Etapa,Unidades,Kilos,Origen,Cod. Origen,Nombre Origen
53983,101004,SOCIEDAD LA BOTICA LTDA.,"ENSENADA NAGLE, ESTE DE PUNTA AUCAN",CERRADO,Mar,2021,1,Chiloé,Ancud,480,OSTRA CHILENA,SEMILLAS,132000.0,1100.0,Otros,1.0,CAPTACION NATURAL
53984,101422,"SANCHEZ MIRANDA, RUBEN SEGUNDO","LECAM, BAHIA LINAO",CERRADO,Mar,2021,1,Chiloé,Ancud,430,CHORITO,SEMILLAS,NaN,1932.0,Otros,6.0,PERMISO DE ESCASA IMPORTANCIA
54005,102109,INVERSIONES COIHUIN LTDA.,BAHIA LINAO,CERRADO,Mar,2021,1,Chiloé,Ancud,430,CHORITO,SEMILLAS,NaN,3000.0,Centro de Cultivo,102810.0,BAEZ MALDONADO JUAN HOMERO
54006,102109,INVERSIONES COIHUIN LTDA.,BAHIA LINAO,CERRADO,Mar,2021,1,Chiloé,Ancud,430,CHORITO,SEMILLAS,NaN,3000.0,Centro de Cultivo,102810.0,BAEZ MALDONADO JUAN HOMERO
54007,102109,INVERSIONES COIHUIN LTDA.,BAHIA LINAO,CERRADO,Mar,2021,1,Chiloé,Ancud,430,CHORITO,SEMILLAS,NaN,4000.0,Centro de Cultivo,102810.0,BAEZ MALDONADO JUAN HOMERO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64047,103763,INVERSIONES COIHUIN LTDA.,"BAHIA HUELMO, ESTE PUNTA CONCHA BLANCA",CERRADO,Mar,2022,6,Llanquihue,Puerto Montt,430,CHORITO,SEMILLAS,NaN,25000.0,Centro de Cultivo,102358.0,INVERSIONES COIHUIN LTDA.
64048,103763,INVERSIONES COIHUIN LTDA.,"BAHIA HUELMO, ESTE PUNTA CONCHA BLANCA",CERRADO,Mar,2022,6,Llanquihue,Puerto Montt,430,CHORITO,SEMILLAS,NaN,20000.0,Área de Manejo,10208.0,Weste Isla Pelada Sector B
64049,104092,INVERSIONES COIHUIN LTDA.,"ESTE PUNTA CAPACHO, BAHIA ILQUE",CERRADO,Mar,2022,6,Llanquihue,Puerto Montt,430,CHORITO,SEMILLAS,NaN,17604.0,Área de Manejo,10207.0,Weste Isla Pelada Sector A
64052,103610,SOCIEDAD ACUI CHOLMAR SPA,"CANAL LLANCAHUE, SURESTE ISLA MUTRI",CERRADO,Mar,2022,6,Palena,Hualaihué,430,CHORITO,SEMILLAS,NaN,96000.0,Otros,1.0,CAPTACION NATURAL


In [ ]:
#ingresos_filtered

RNA_list_CCAA = RNA_filtered['Codigo Centro'].unique().tolist()
reporte_plantas_list_origen = reporte_plantas_filtered['Código Origen'].unique().tolist()

In [ ]:
results = {}
for i in RNA_list_CCAA:
    results[i] = reporte_plantas_list_origen.count(i) 

In [ ]:
df_RNA_reporte_plantas = pd.DataFrame.from_dict(results,orient='index', columns=['count'])
#type(df_RNA_reporte_plantas['count'])
ccaa_match_with_reporte = df_RNA_reporte_plantas[df_RNA_reporte_plantas['count'] > 0]
ccaa_match_with_reporte


,count
100022,1
100242,1
100339,1
101000,1
101011,1
...,...
104342,1
104347,1
104349,1
104366,1


In [ ]:
join_RNA_reportes = pd.merge(RNA_filtered.rename({'user_id': 'user_id_r'}, reporte_plantas_filtered, how='left', left_on='Codigo Centro', right_on='Código Origen'))
join_RNA_reportes

,Región,Sector,Nm_Provincia,Comuna Centro,Nombre Area,Codigo Centro,Nombre Centro,Categoria,N° Folio RNA,Rut,...,utilizado,Toneladas Rest,Unidades Rest,Tipo Declaración,idBodega,Nombre Bodega,Tipo Bodega,Dirección Bodega,Ciudad Bodega,Región Bodega
0,10,RÌO PUDETO,Chiloé,Ancud,NaN,100009,*,INSCRITO,624.0,84913200-8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,"ESTERO CHOPE, ISLA PULUQUI, SECTOR 1",Llanquihue,Calbuco,NaN,100019,*,INSCRITO,6981.0,69220600-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,CALETA RILAN,Chiloé,Castro,NaN,100022,ENSENADA RILAN,INSCRITO,3162.0,84764200-9,...,-1,0.0,-,Planta Abastecimiento,2719.0,PATIO,PLANTA,AUCAR SN,Ancud,X REGION
3,10,CALETA RILAN,Chiloé,Castro,NaN,100022,ENSENADA RILAN,INSCRITO,3162.0,84764200-9,...,-1,0.0,-,Planta Abastecimiento,2719.0,PATIO,PLANTA,AUCAR SN,Ancud,X REGION
4,10,CALETA RILAN,Chiloé,Castro,NaN,100022,ENSENADA RILAN,INSCRITO,3162.0,84764200-9,...,-1,0.0,-,Planta Abastecimiento,2719.0,PATIO,PLANTA,AUCAR SN,Ancud,X REGION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53720,10,AAMERB BALTAZAR,Palena,Hualaihué,NaN,810248,AAMERB - BALTAZAR,INSCRITO,0.0,65305390-8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53721,10,"PASO QUILOLCURA, NORTE PUNTA LILECURA, ISLA QUENU",Llanquihue,Calbuco,NaN,810262,AAMERB - ISLA QUENU SECTOR B,INSCRITO,0.0,65091450-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53722,10,ESTERO PICHICOLO,Palena,Hualaihué,NaN,810276,AAMERB - ESTE PUNTILLA PICHICOLU,INSCRITO,0.0,65260290-8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53723,10,"CANAL DESERTORES, NORTE CALETA CHANA",Palena,Chaitén,NaN,810286,AAMERB - CHANA SECTOR B,INSCRITO,0.0,74053200-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
